In [3]:
%use kotlin-statistics, krangl, kravis

In [4]:
import kotlin.math.*

In [5]:
@file:Repository("https://dl.bintray.com/kyonifer/maven")
@file:DependsOn("com.kyonifer:koma-core-ejml:0.12")
@file:DependsOn("com.kyonifer:koma-plotting:0.12")

In [6]:
import koma.extensions.*
import koma.*

In [7]:
%use lets-plot

In [8]:
fun trainTestSplit(path: String, fraction: Double = 0.8): Pair<DataFrame, DataFrame> {
    val df = DataFrame.readTSV(path)
    val trainCount: Int = kotlin.math.ceil(fraction * df.nrow).toInt()
    val shuffledDf = df.shuffle()
    return Pair(shuffledDf.slice(0..trainCount), shuffledDf.slice(trainCount + 1..shuffledDf.nrow))
}

In [20]:
class BoW(private val limit: Int = 100) {

    public var voc: Map<String, Int> = HashMap()

    fun initialize(df: DataFrame) {
        val events = df["events"]
        val eventsMap: MutableMap<String, Int> = HashMap()
        for (i in 0 until events.length) {
            for (ev in events[i].toString().split(" , ")) {
                if (eventsMap.containsKey(ev)) {
                    eventsMap[ev] = eventsMap.getValue(ev) + 1
                } else {
                    eventsMap[ev] = 1
                }
            }
        }
        val realLimit = if (eventsMap.size > limit) limit else eventsMap.size
        voc = eventsMap.toSortedMap(compareBy({ eventsMap[it]?.times(-1) }, { it })).keys.toList()
            .slice(0 until realLimit).withIndex().toList().associate {it.value to it.index}
    }

    fun transform(df: DataFrame): Pair<koma.matrix.Matrix<Double>, List<String>> {
        val mat = zeros(df.nrow, voc.size + 2)
        val label = ArrayList<String>()
        for (i in 0 until df.nrow) {
            mat[i, voc.size] = df["min"][i] as Int
            mat[i, voc.size + 1] = df["events"][i].toString().split(" , ").size
            label.add(df["category"][i].toString())
            for (event in df["events"][i].toString().split(" , ")) {
                if (voc.containsKey(event)) {
                    mat[i, voc[event] as Int] += 1
                }
            }
        }
        return Pair(mat, label)
    }
    
    fun transform2(df: DataFrame): koma.matrix.Matrix<Double> {
        val mat = zeros(df.nrow, voc.size + 2)
        for (i in 0 until df.nrow) {
            mat[i, voc.size] = df["min"][i] as Int
            mat[i, voc.size + 1] = df["events"][i].toString().split(" , ").size
            for (event in df["events"][i].toString().split(" , ")) {
                if (voc.containsKey(event)) {
                    mat[i, voc[event] as Int] += 1
                }
            }
        }
        return mat
    }

}

In [10]:
class NaiveBayesCounter(private val alpha: Double = 1e-5) {

    private var classProb: MutableMap<String, Double> = HashMap()
    private var classes: MutableList<String> = ArrayList()
    private var probs: MutableMap<String, koma.matrix.Matrix<Double>> = HashMap()


    fun fit(X: koma.matrix.Matrix<Double>, y: List<String>) {
        for (i in y.indices) {
            for (cls in y[i].toLowerCase().split(", ")) {
                if (!classProb.containsKey(cls)) {
                    classProb[cls] = 1.0 / y.size
                    classes.add(cls)
                    probs[cls] = ones(X.shape()[1] - 2, 11) * alpha
                } else {
                    classProb[cls] = classProb.getValue(cls) + 1.0 / y.size
                }
                for (j in 0 until X.shape()[1] - 2) {
                    val ind:Int = if (X[i, j] < 10.0) X[i, j].toInt() else 10
                    probs.getOrDefault(cls, zeros(0, 0))[j, ind] += 1
                }
            }

        }
        for (cls in classes) {
            var probsMatrix = probs.getOrDefault(cls, zeros(0, 0))
            val den = probsMatrix.getRow(0).elementSum().toInt()
            probsMatrix /= den
            probs[cls] = probsMatrix
        }
    }




    fun logProb(X: koma.matrix.Matrix<Double>): koma.matrix.Matrix<Double> {
        val ans = zeros(X.shape()[0], classes.size)
        for (i in 0 until X.shape()[0]) {
            for (j in classes.indices) {
                val cls = classes[j]
                ans[i, j] = kotlin.math.ln(classProb.getOrDefault(cls, alpha))
                for (k in 0 until X.shape()[1] - 2) {
                    val ind = if (X[i, k] < 10.0) X[i, k].toInt() else 10
                    ans[i, j] += kotlin.math.ln(probs.getOrDefault(cls, zeros(0, 0))[k, ind])
                }
            }
        }
        return ans
    }

    fun predict(X: koma.matrix.Matrix<Double>): List<String> {
        val probs = logProb(X)

        val ans = ArrayList<String>()

        probs.forEachRow { ans.add(classes[it.argMax()]) }
        return ans
    }
}

In [11]:
val (train, test) = trainTestSplit("../data/sh_sessions_multi.tsv")

In [11]:
fun testPrecisionRecall(cat: String="coding"): Triple<ArrayList<Double>, ArrayList<Double>, ArrayList<Int>> {
    val rec = ArrayList<Double>()
    val prec = ArrayList<Double>()
    val x = ArrayList<Int>()
    for (i in 10 until 450 step 10) {
        val bow = BoW(i)
        bow.initialize(train)

        val (X_bow, y_bow) = bow.transform(train)
        val (X_bow_test, y_bow_test) = bow.transform(test)
        val classificator = NaiveBayesCounter(1e-4)
        classificator.fit(X_bow, y_bow)

        val ans = classificator.predict(X_bow_test)
        var TP = 0.0
        var FN = 0.0
        var FP = 0.0

        for(i in y_bow_test.indices) {

            if (y_bow_test[i].toLowerCase().contains(cat) && ans[i].contains(cat)) {
                TP += 1
            }

            if (y_bow_test[i].toLowerCase().contains(cat) && !ans[i].contains(cat)) {
                FN += 1
            }

            if (!y_bow_test[i].toLowerCase().contains(cat) && ans[i].contains(cat)) {
                FP += 1
            }
        }
        rec.add(TP / (TP + FN))
        prec.add(TP / (TP + FP))


        x.add(i)
    }
    return Triple(rec, prec, x)
}

In [12]:
val res = ArrayList<Double>()
val x = ArrayList<Int>()

In [13]:
for (i in 10 until 400 step 10) {
    val bow = BoW(i)
    bow.initialize(train)

    val (X_bow, y_bow) = bow.transform(train)
    val (X_bow_test, y_bow_test) = bow.transform(test)
    val classificator = NaiveBayesCounter(1e-4)
    classificator.fit(X_bow, y_bow)

    val ans = classificator.predict(X_bow_test)
    var tot = 0.0
    var TP = 0.0
    var FN = 0.0
    var FP = 0.0

    for(i in y_bow_test.indices) {
        if (y_bow_test[i].toLowerCase().contains(ans[i])) {
            tot += 1
        }
    }
    res.add(tot / y_bow_test.size)
    x.add(i)
}

In [51]:
val test = DataFrame.readTSV("../data/test.tsv")
val train = DataFrame.readTSV("../data/sh_sessions_multi.tsv")

In [73]:
train

session_id start_time sec min category events 1305192621b4643-a1d4-4288-9cb2-9ece426a01f1_1 26.08.2019 10:28 164 2 Coding, VCS <td style="text-align:left" title="actions_Find , actions_com.intellij.find.SearchReplaceComponent$10 , actions_com.intellij.openapi.fileEditor.impl.EditorTabbedContainer$CloseTab , toolwindow_Project , actions_FindInPath , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , actions_com.intellij.find.impl.FindPopupPanel$MyEnterAction , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , file.types.usage_JAVA , actions_GotoDeclaration , productivity_navigation.goto.declaration , file.types.usage_JAVA , productivity_editing.select.word , actions_Find , actions_NextOccurence , actions_com.intellij.find.SearchReplaceComponent$10 , actions_GotoDeclaration , productivity_navigation.goto.declaration , productivity_editing.select.word , actions_EditorCopy , actions_Back , actions_Vcs.ShowTabbedFileHistory , toolwindow_Version_Control , toolbar_Vcs.ShowTabbedFileHistory , actions_Diff.ShowDiff , actions_HideActiveWindow , toolbar_HideActiveWindow , actions_FindInPath , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , actions_GotoDeclaration , productivity_navigation.goto.declaration , file.types.usage_JAVA , actions_GotoDeclaration , productivity_navigation.goto.declaration , file.types.usage_JAVA , gutter.icon.click_implementedMethod , actions_com.intellij.codeInsight.daemon.NavigateAction , file.types.usage_JAVA , actions_Back , productivity_editing.select.word , actions_GotoDeclaration , productivity_navigation.goto.declaration , file.types.usage_JAVA , actions_com.intellij.codeInsight.daemon.NavigateAction , gutter.icon.click_implementedMethod , file.types.usage_JAVA , productivity_editing.select.word , actions_GotoDeclaration , productivity_navigation.goto.declaration , file.types.usage_JAVA , actions_EditorCopy">actions_Find , actions_com.intellij.find.Search... 0104192bd9b36a1-98c7-4a5e-817d-313d69be0ba3_1 30.08.2019 20:16 66 1 Coding actions_FindInPath , toolwindow_Find , ui.dialo... 0304192f4244c10-88fc-4a38-992b-28bf8ffcfb58_0 27.08.2019 6:18 11 0 Ide start, Ide close event.log_whitelist.loaded , lifecycle_ide.star... 0905192c46ec682-3f47-4af1-b313-755dbfcfe1fa_4 27.08.2019 11:24 748 12 Coding toolwindow_Project , file.types.usage_JAVA , ac... 1404192054d7ef6-857b-4fd3-9084-333d5eb67b73_27 30.08.2019 10:04 489 8 Coding <td style="text-align:left" title="productivity_SearchEverywhere , searchEverywhere_dialogOpen , actions_GotoClass , actions_EditorPaste , actions_third.party , file.types.usage_JAVA , actions_Find , actions_EditorPaste , actions_com.intellij.find.SearchReplaceComponent$10 , productivity_editing.select.word , productivity_editing.select.word , actions_CallHierarchy , toolwindow_Hierarchy , file.types.usage_JAVA , productivity_editing.select.word , actions_EditorCopy , actions_validation.unmatched_rule , actions_EditorPaste , productivity_navigation.popup.camelprefix , file.types.usage_JAVA , productivity_editing.select.word , actions_Back , productivity_editing.select.word , actions_validation.unmatched_rule , productivity_navigation.popup.camelprefix , productivity_editing.select.word , actions_GotoImplementation , productivity_navigation.goto.implementation , file.types.usage_JAVA , actions_FindInPath , find_FindInPath.FileFilterOn , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , actions_EditorPaste , find_FindPopupPanel$ToggleFileFilter , ui.dialogs_com.intellij.find.impl.FindPopupPanel$2 , actions_CallHierarchy , toolwindow_Hierarchy , productivity_editing.select.word , actions_GotoDeclaration , productivity_navigation.goto.declaration , productivity_editing.select.word , actions_validation.unmatched_rule , productivity_navigation.popup.camelprefix , actions_GotoDeclaration , productivity_navigation.goto.declaration , productivity_editing.select.word , productivity_editing.select.word , actions_FindInPath , ui.dialogs_

In [52]:
val bow = BoW(300)
bow.initialize(train)

In [53]:
val (X_bow, y_bow) = bow.transform(train)

In [54]:
val X_bow_test = bow.transform2(test)
val classificator = NaiveBayesCounter(1e-4)
classificator.fit(X_bow, y_bow)

In [55]:
val ans = classificator.predict(X_bow_test)

In [56]:
val ans_c = StringCol("type", ans)

In [57]:
val ndf = test.addColumn("type") { ans_c }

In [67]:
ndf.filterByRow {it["type"].toString() == "ide close"}

session_id min events type 2204192da927d8d-1396-4efd-8e1a-18b59ebf81c6_6 0 lifecycle_ide.close , lifecycle_project.closed ide close 15051928161ddca-78a7-4b72-9326-02556598a649_64 0 lifecycle_ide.close , lifecycle_project.closed ... ide close 15051928161ddca-78a7-4b72-9326-02556598a649_65 3 <td style="text-align:left" title="event.log_whitelist.loaded , lifecycle_ide.start , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , lifecycle_project.opened , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , lifecycle_project.opened , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , lifecycle_project.opening.finished , lifecycle_project.opening.finished , startup_totalDuration , lifecycle_ide.close , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , lifecycle_project.closed , lifecycle_project.closed">event.log_whitelist.loaded , lifecycle_ide.star... ide close 20121913aefe0d2-b6e1-4b9a-bd6d-54d4999fe1df_19 2 actions_ToggleLineBreakpoint , debugger.breakpo... ide close 2112191448c97a7-114e-4827-93de-3363865f3f42_65 0 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 2112191448c97a7-114e-4827-93de-3363865f3f42_66 1 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 10022018d5b7303-531b-4844-9bc9-c19da4c3d20e_14 1 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 2603191d7d70721-2b6b-4957-a81b-76b7a366eac2_3 0 lifecycle_ide.close , lifecycle_project.closed ide close 2603191d7d70721-2b6b-4957-a81b-76b7a366eac2_7 0 lifecycle_ide.close , lifecycle_project.closed ide close 19121928e3df98c-4369-4612-9838-656516df0f6c_5 0 lifecycle_project.closed , lifecycle_ide.close ide close 2910191b7c39ee2-b434-4b8b-a6ba-9dd5d754c7f3_13 0 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 2910191b7c39ee2-b434-4b8b-a6ba-9dd5d754c7f3_16 0 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 28031932bbc9e8d-be1f-485f-91d9-1de5da541863_6 0 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close 0203203b4ea4636-ef2a-4a38-8999-e76cd6206b44_132 4 lifecycle_project.closed , lifecycle_project.cl... ide close 26041913ff32878-e549-40d8-a928-dabbaa07d749_5 0 <td style="text-align:left" title="ide.self.update_dialog.update.started , lifecycle_ide.close , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , ui.dialogs_com.intellij.openapi.progress.util.ProgressDialog$MyDialogWrapper , lifecycle_project.closed">ide.self.update_dialog.update.started , lifecyc... ide close 26041913ff32878-e549-40d8-a928-dabbaa07d749_7 0 lifecycle_project.closed , lifecycle_ide.close ... ide close 05091917988b662-be79-4900-aa4f-25c6538676c5_37 1 lifecycle_project.closed , ui.dialogs_com.intel... ide close 24012027effc16e-22cb-4141-8b63-8c1d30ec8c4b_1 0 lifecycle_ide.close , ui.dialogs_com.intellij.o... ide close 24012027effc16e-22cb-4141-8b63-8c1d30ec8c4b_5 1 actions_Stop , toolbar_Stop , run.configuration... ide close 2411191c94b3bf9-6356-40c8-850b-5a4236a9836c_3 0 ui.dialogs_com.intellij.openapi.ui.messages.Mes... ide close ... only showing top 20 rows

In [68]:
val r = ndf.filterByRow {it["type"].toString() == "ide close"}["events"]

In [72]:
r[4]

ui.dialogs_com.intellij.openapi.ui.messages.MessageDialog , run.configuration.exec_finished , lifecycle_project.closed

In [14]:
val df = mapOf("x" to x, "y" to res )
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [15]:
var (rec, prec, x) = testPrecisionRecall("coding")

In [16]:
val df = mapOf("x" to x, "y" to prec)
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [17]:
val df = mapOf<String, Any>("x" to x, "y" to rec )
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [18]:
var (rec, prec, x) = testPrecisionRecall("ide start")

In [19]:
val df = mapOf("x" to x, "y" to prec)
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [20]:
val df = mapOf("x" to x, "y" to rec)
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [21]:
var (rec, prec, x) = testPrecisionRecall("ide close")

In [22]:
val df = mapOf("x" to x, "y" to rec)
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()

In [23]:
val df = mapOf("x" to x, "y" to prec)
val p = lets_plot(df) {x = "x"; y = "y"}
p + geom_line()